# Learning Holomorphic Volume Form with Neural Network

This notebook trains a feedforward neural network to learn the holomorphic volume form (Omega) from CY point coordinates.

**Goal:** Learn the real and imaginary parts of the complex holomorphic volume form components.

**Steps:**
1. Setup and generate CY points
2. Compute holomorphic volume form and Hermitian metric
3. Prepare training data
4. Train neural network
5. Test on independent sample

## 1. Import Libraries

In [1]:
# Import standard libraries
import os
import sys
import yaml
import numpy as np
import tensorflow as tf
import pickle
import pathlib
import matplotlib.pyplot as plt

# Setup path for cymetric package (from scratch/ to github/)
_parent_dir = pathlib.Path(os.getcwd()).parent
_cymetric_dir = _parent_dir.parent / "cymetric"
if str(_parent_dir) not in sys.path:
    sys.path.insert(0, str(_parent_dir))
if str(_cymetric_dir) not in sys.path:
    sys.path.insert(0, str(_cymetric_dir))

# Create alias to fix cymetric internal imports
import cymetric
if hasattr(cymetric, 'cymetric'):
    sys.modules['cymetric'] = cymetric.cymetric

# Import geometry functions
from geometry.patches import CoordChange_C5R10

# Import cymetric functions
from cymetric.pointgen.pointgen import PointGenerator
from cymetric.models.tfhelper import prepare_tf_basis
from cymetric.models.tfmodels import PhiFSModel

print("✅ All imports successful!")

✅ All imports successful!


## 2. Load Configuration

In [2]:
# Set up paths (go to parent directory)
cymodel_name = ''  # Set to '_name' if using named model
parent_dir = os.path.dirname(os.getcwd())
dirname = os.path.join(parent_dir, 'models', 'cy_models', 'link_data')
config_path = os.path.join(parent_dir, f'models/cy_models/cy_model_config{cymodel_name}.yaml')
cymodel_path = os.path.join(parent_dir, f'models/cy_models/cy_metric_model{cymodel_name}.keras')

# Load configuration
with open(config_path, 'r') as f:
    config = yaml.unsafe_load(f)

print("Configuration loaded:")
print(f"  Monomials shape: {np.array(config['monomials']).shape}")
print(f"  Ambient dimension: {config['ambient']}")

Configuration loaded:
  Monomials shape: (5, 5)
  Ambient dimension: [4]


## 3. Generate Training Data

In [3]:
# Generate training points
n_train = 5000  # Number of training points

print(f"Generating {n_train} training CY points...")

# Initialize point generator
pg = PointGenerator(
    config['monomials'], 
    config['coefficients'], 
    config['kmoduli'], 
    config['ambient']
)

# Generate dataset
kappa = pg.prepare_dataset(n_train, dirname, val_split=0.)
pg.prepare_basis(dirname, kappa=kappa)

# Load generated data
data = np.load(os.path.join(dirname, 'dataset.npz'))
points_train = data['X_train']

# Convert to C^5 complex coordinates
cy_points_train = CoordChange_C5R10(points_train, inverse=True)

# Identify patch indices
one_idxs_train = np.argmax(np.isclose(cy_points_train, complex(1, 0)), axis=1)
dropped_idxs_train = pg._find_max_dQ_coords(cy_points_train)

# Extract C^3 coordinates (excluding one_idx and dropped_idx)
mask_train = np.ones(cy_points_train.shape, dtype=bool)
samples_train = np.arange(cy_points_train.shape[0])
mask_train[samples_train, one_idxs_train] = False
mask_train[samples_train, dropped_idxs_train] = False
c3_coords_train = cy_points_train[mask_train].reshape(cy_points_train.shape[0], -1)

print(f"✅ Training data generated:")
print(f"  Points in R^10: {points_train.shape}")
print(f"  Points in C^5: {cy_points_train.shape}")
print(f"  Points in C^3 (local coords): {c3_coords_train.shape}")

Generating 5000 training CY points...


pointgen:INFO:Vol_k: 5.0, Vol_cy: 280.4873248050895.


✅ Training data generated:
  Points in R^10: (5000, 10)
  Points in C^5: (5000, 5)
  Points in C^3 (local coords): (5000, 3)


## 4. Compute Holomorphic Volume Form and Hermitian Metric

In [ ]:
# Compute holomorphic volume form for training data
holomorphic_volume_form_train = pg.holomorphic_volume_form(cy_points_train)

print(f"Holomorphic volume form (training):")
print(f"  Shape: {holomorphic_volume_form_train.shape}")
print(f"  Dtype: {holomorphic_volume_form_train.dtype}")
print(f"  This is a complex (3,0)-form on the CY")

In [ ]:
# Load basis and create CyMetric model
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS_tf = prepare_tf_basis(BASIS)

# Build neural network architecture
nn_phi = tf.keras.Sequential()
nn_phi.add(tf.keras.Input(shape=(config['n_in'],)))
for _ in range(config['nlayer']):
    nn_phi.add(tf.keras.layers.Dense(config['nHidden'], activation=config['act']))
nn_phi.add(tf.keras.layers.Dense(config['n_out'], use_bias=False))

# Create PhiFSModel
cymetric_model = PhiFSModel(nn_phi, BASIS_tf, alpha=config['alpha'])
cymetric_model.nn_phi = tf.keras.models.load_model(cymodel_path)

# Compute Hermitian metric
hermitian_metric_train = cymetric_model(CoordChange_C5R10(cy_points_train)).numpy()
hermitian_metric_train = 0.5 * (hermitian_metric_train + hermitian_metric_train.conj().transpose(0, 2, 1))

print(f"✅ CyMetric model loaded")
print(f"Hermitian metric (training):")
print(f"  Shape: {hermitian_metric_train.shape}")

## 5. Prepare Neural Network Training Data

Extract the complex components of the holomorphic volume form and convert to real components for learning.

In [ ]:
# The holomorphic volume form has only one independent component: Omega_{012}
# Extract it
print(f"Omega_{{012}} components:")
print(f"  Shape: {holomorphic_volume_form_train.shape}")
print(f"  Sample values: {holomorphic_volume_form_train[:3]}")

# Convert to real and imaginary parts for training targets
omega_real_train = np.real(holomorphic_volume_form_train)
omega_imag_train = np.imag(holomorphic_volume_form_train)

# Stack as [real, imag] for each point
y_train = np.stack([omega_real_train, omega_imag_train], axis=1)

print(f"\nTraining targets (real and imag parts):")
print(f"  Shape: {y_train.shape}")
print(f"  [real, imag] for first 3 points:")
print(y_train[:3])

## 6. Prepare Input Features

Choose between:
- **Option A:** R^10 coordinates (embedding space) from `points_train`
- **Option B:** R^16 coordinates: C^3 local coordinates (as R^6) + one-hot encoded patch indices (10 dims: 5 + 5)

In [ ]:
# Choose input type
INPUT_TYPE = 'c3'  # Options: 'r10' or 'c3'

if INPUT_TYPE == 'r10':
    # Use R^10 embedding coordinates
    X_train = points_train
    input_dim = 10
    print(f"Using R^10 embedding coordinates")
elif INPUT_TYPE == 'c3':
    # Use C^3 local coordinates as R^6 (real and imaginary parts) + one-hot encoded patch indices
    # One-hot encode patch indices (5 classes each)
    one_idx_onehot_train = tf.keras.utils.to_categorical(one_idxs_train, num_classes=5)
    dropped_idx_onehot_train = tf.keras.utils.to_categorical(dropped_idxs_train, num_classes=5)
    
    X_train = np.concatenate([
        np.real(c3_coords_train), 
        np.imag(c3_coords_train),
        one_idx_onehot_train,
        dropped_idx_onehot_train
    ], axis=1)
    input_dim = 16  # 6 (coords) + 5 (one_idx) + 5 (dropped_idx)
    print(f"Using C^3 local coordinates (as R^6) + one-hot encoded patch indices (10)")
    print(f"  Components: [Re(z1-3), Im(z1-3), one_hot(one_idx), one_hot(dropped_idx)]")
    print(f"  Input dimension: 6 coords + 5 one_idx + 5 dropped_idx = 16")
else:
    raise ValueError(f"Unknown INPUT_TYPE: {INPUT_TYPE}")

print(f"\nInput features:")
print(f"  Shape: {X_train.shape}")
print(f"  Input dimension: {input_dim}")

## 7. Generate Test Data (Independent Sample)

In [ ]:
# Generate test points
n_test = 20000

print(f"Generating {n_test} test CY points...")

# Generate new dataset for testing
kappa_test = pg.prepare_dataset(n_test, dirname, val_split=0.)
data_test = np.load(os.path.join(dirname, 'dataset.npz'))
points_test = data_test['X_train']

# Convert to C^5 complex coordinates
cy_points_test = CoordChange_C5R10(points_test, inverse=True)

# Identify patch indices
one_idxs_test = np.argmax(np.isclose(cy_points_test, complex(1, 0)), axis=1)
dropped_idxs_test = pg._find_max_dQ_coords(cy_points_test)

# Extract C^3 coordinates
mask_test = np.ones(cy_points_test.shape, dtype=bool)
samples_test = np.arange(cy_points_test.shape[0])
mask_test[samples_test, one_idxs_test] = False
mask_test[samples_test, dropped_idxs_test] = False
c3_coords_test = cy_points_test[mask_test].reshape(cy_points_test.shape[0], -1)

# Compute holomorphic volume form for test data
holomorphic_volume_form_test = pg.holomorphic_volume_form(cy_points_test)
omega_real_test = np.real(holomorphic_volume_form_test)
omega_imag_test = np.imag(holomorphic_volume_form_test)
y_test = np.stack([omega_real_test, omega_imag_test], axis=1)

# Prepare test input features
if INPUT_TYPE == 'r10':
    X_test = points_test
elif INPUT_TYPE == 'c3':
    # One-hot encode test patch indices
    one_idx_onehot_test = tf.keras.utils.to_categorical(one_idxs_test, num_classes=5)
    dropped_idx_onehot_test = tf.keras.utils.to_categorical(dropped_idxs_test, num_classes=5)
    
    X_test = np.concatenate([
        np.real(c3_coords_test), 
        np.imag(c3_coords_test),
        one_idx_onehot_test,
        dropped_idx_onehot_test
    ], axis=1)

print(f"✅ Test data generated:")
print(f"  X_test shape: {X_test.shape}")
print(f"  y_test shape: {y_test.shape}")

## 8. Build and Train Neural Network

### Hyperparameters (Edit these!)

## 💡 Suggestions for Improving Learning Performance

If the model is performing poorly, try these approaches:

### **1. Input Normalization/Standardization**
- **Problem**: Input features may have very different scales
- **Solution**: Add standardization of inputs (z-score normalization)
- **Implementation**: Use `StandardScaler` or normalize manually: `X_norm = (X - X.mean()) / X.std()`
- **Expected Impact**: HIGH - Often critical for neural networks

### **2. Output Normalization**
- **Problem**: Target values may have extreme ranges or be very small/large
- **Solution**: Normalize outputs to [0,1] or standardize them
- **Implementation**: Scale y_train before training, inverse transform predictions
- **Expected Impact**: HIGH - Especially if target values are very small

### **3. Network Architecture**
- **Problem**: Network may be too small/large or wrong shape
- **Options**:
  - Increase depth: `[256, 256, 128, 64, 32]` (more layers for complex functions)
  - Increase width: `[512, 256, 128]` (wider layers)
  - Add residual connections (requires custom implementation)
- **Expected Impact**: MEDIUM - Depends on problem complexity

### **4. Activation Functions**
- **Problem**: Current activation (gelu) may not suit this problem
- **Options**:
  - `'tanh'` - Good for bounded outputs, smooth gradients
  - `'elu'` - Helps with vanishing gradients
  - `'swish'` - Modern alternative to ReLU
  - `'selu'` - Self-normalizing, but requires specific initialization
- **Expected Impact**: LOW-MEDIUM

### **5. Learning Rate & Optimizer**
- **Problem**: LR may be too high/low, or optimizer suboptimal
- **Options**:
  - Lower LR: `0.0001` or `0.00001`
  - Higher LR with warmup: Start at `0.0001`, increase to `0.001`
  - Try different optimizer: `AdamW` (better weight decay)
  - Use cosine annealing: `CosineAnnealingScheduler`
- **Expected Impact**: MEDIUM-HIGH

### **6. Batch Normalization**
- **Problem**: Internal covariate shift slowing learning
- **Solution**: Set `USE_BATCH_NORM = True`
- **Expected Impact**: MEDIUM - Helps with training stability

### **7. Regularization Tuning**
- **Problem**: Underfitting (too much reg) or overfitting (too little)
- **Options**:
  - Reduce L2: Try `0.0001` or `0.00001` or `0.0`
  - Reduce Dropout: `0.0` or `0.001` (current 0.01 may be too high)
  - Try L1 regularization for sparsity
- **Expected Impact**: MEDIUM

### **8. More Training Data**
- **Problem**: 50k samples may not be enough
- **Solution**: Increase `n_train` to `100000` or `200000`
- **Expected Impact**: MEDIUM-HIGH - More data usually helps

### **9. Data Augmentation**
- **Problem**: Limited diversity in training samples
- **Solution**: Add noise to inputs during training, or generate more diverse points
- **Expected Impact**: MEDIUM

### **10. Loss Function**
- **Problem**: MSE may not be optimal for this problem
- **Options**:
  - Try `'mae'` (less sensitive to outliers)
  - Try `'huber'` (robust to outliers)
  - Custom loss: Log-cosh, or weighted MSE
- **Expected Impact**: LOW-MEDIUM

### **11. Complex-Valued Network**
- **Problem**: Learning real/imag separately loses phase information
- **Solution**: Use complex-valued neural network layers (requires custom implementation)
- **Expected Impact**: HIGH - But requires significant code changes

### **12. Physics-Informed Constraints**
- **Problem**: Network doesn't respect geometric properties
- **Solution**: Add loss terms for known constraints (e.g., |Omega|² should match expectations)
- **Expected Impact**: HIGH - Encodes domain knowledge

### **Recommended Priority Order:**
1. **START HERE**: Input/Output normalization (#1, #2) ⭐⭐⭐
2. Lower learning rate and remove/reduce dropout (#5, #7)
3. Increase training data (#8)
4. Try different architecture (#3)
5. Enable batch normalization (#6)
6. Experiment with loss functions and activation (#10, #4)

### **Quick Win Combination:**
```python
# Normalize inputs and outputs
X_train_mean, X_train_std = X_train.mean(axis=0), X_train.std(axis=0)
X_train_norm = (X_train - X_train_mean) / (X_train_std + 1e-8)
X_test_norm = (X_test - X_train_mean) / (X_train_std + 1e-8)

y_train_mean, y_train_std = y_train.mean(axis=0), y_train.std(axis=0)
y_train_norm = (y_train - y_train_mean) / (y_train_std + 1e-8)
# Train on normalized data, then: y_pred_real = y_pred_norm * y_train_std + y_train_mean

# Adjust hyperparameters
HIDDEN_LAYERS = [256, 256, 128, 64]
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.0
USE_BATCH_NORM = True
L2_REG = 0.0001
```

In [ ]:
# ============================================================================
# HYPERPARAMETERS - Edit these to tune the model
# ============================================================================

# Data preprocessing
NORMALIZE_INPUTS = True        # Standardize input features (z-score normalization)
NORMALIZE_OUTPUTS = True       # Standardize output targets (z-score normalization)

# Architecture
HIDDEN_LAYERS = [256, 256, 128, 64]  # List of hidden layer sizes
ACTIVATION = 'gelu'            # Activation function: 'relu', 'gelu', 'tanh', 'elu'
OUTPUT_ACTIVATION = None       # Output activation (None for linear)
USE_BATCH_NORM = True         # Use batch normalization between layers
DROPOUT_RATE = 0.01             # Dropout rate (0.0 = no dropout)

# Training
LEARNING_RATE = 0.0002          # Initial learning rate
BATCH_SIZE = 128                # Batch size for training
EPOCHS = 200                   # Maximum number of epochs
VALIDATION_SPLIT = 0.1         # Fraction of training data for validation

# Regularization
L2_REG = 0.001                # L2 regularization coefficient (for layers)
WEIGHT_DECAY = 0.001            # Weight decay for AdamW optimizer

# Callbacks
EARLY_STOPPING_PATIENCE = 100   # Stop if no improvement for this many epochs
REDUCE_LR_PATIENCE = 15        # Reduce LR if no improvement for this many epochs
REDUCE_LR_FACTOR = 0.5         # Factor to reduce LR by

# Loss function
LOSS = 'huber'                   # 'mse', 'mae', 'huber'

# ============================================================================

print("Hyperparameters set:")
print(f"  Normalize inputs: {NORMALIZE_INPUTS}")
print(f"  Normalize outputs: {NORMALIZE_OUTPUTS}")
print(f"  Architecture: {input_dim} -> {' -> '.join(map(str, HIDDEN_LAYERS))} -> 2")
print(f"  Activation: {ACTIVATION}")
print(f"  Batch normalization: {USE_BATCH_NORM}")
print(f"  Dropout: {DROPOUT_RATE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Weight decay: {WEIGHT_DECAY}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Max epochs: {EPOCHS}")
print(f"  Loss: {LOSS}")

### Normalize Data (Optional)

In [ ]:
# Normalize inputs and outputs based on flags
if NORMALIZE_INPUTS:
    if INPUT_TYPE == 'r10':
        # R^10: Normalize all features
        X_train_mean = X_train.mean(axis=0)
        X_train_std = X_train.std(axis=0) + 1e-8
        
        X_train_normalized = (X_train - X_train_mean) / X_train_std
        X_test_normalized = (X_test - X_train_mean) / X_train_std
        
        print("✅ Input normalization applied (R^10):")
        print(f"  All 10 dimensions normalized")
        
    elif INPUT_TYPE == 'c3':
        # C^3: Only normalize continuous coordinate features (first 6 dims)
        # Keep one-hot encoded patch indices sparse (last 10 dims)
        X_train_coords = X_train[:, :6]  # C^3 real and imaginary parts
        X_train_patches = X_train[:, 6:]  # One-hot patches (keep sparse)
        
        X_test_coords = X_test[:, :6]
        X_test_patches = X_test[:, 6:]
        
        # Normalize only coordinates
        X_coords_mean = X_train_coords.mean(axis=0)
        X_coords_std = X_train_coords.std(axis=0) + 1e-8
        
        X_train_coords_norm = (X_train_coords - X_coords_mean) / X_coords_std
        X_test_coords_norm = (X_test_coords - X_coords_mean) / X_coords_std
        
        # Recombine: normalized coords + unnormalized sparse one-hot patches
        X_train_normalized = np.concatenate([X_train_coords_norm, X_train_patches], axis=1)
        X_test_normalized = np.concatenate([X_test_coords_norm, X_test_patches], axis=1)
        
        print("✅ Input normalization applied (C^3 + patches):")
        print(f"  Coordinates (dims 0-5) normalized: mean≈0, std≈1")
        print(f"  Patch indices (dims 6-15) kept sparse: values in {{0, 1}}")
        print(f"  Normalized coords - mean: {X_train_coords_norm.mean(axis=0)[:3]}")
        print(f"  Sparse patches - sample: {X_train_patches[0]}")
        
    print(f"  Original X_train shape: {X_train.shape}")
    print(f"  Normalized X_train shape: {X_train_normalized.shape}")
else:
    X_train_normalized = X_train
    X_test_normalized = X_test
    print("ℹ️  Input normalization: DISABLED")

if NORMALIZE_OUTPUTS:
    # Compute normalization statistics from training targets
    y_train_mean = y_train.mean(axis=0)
    y_train_std = y_train.std(axis=0) + 1e-8
    
    # Normalize training and test outputs
    y_train_normalized = (y_train - y_train_mean) / y_train_std
    y_test_normalized = (y_test - y_train_mean) / y_train_std
    
    print("\n✅ Output normalization applied:")
    print(f"  Original y_train - mean: {y_train.mean(axis=0)}, std: {y_train.std(axis=0)}")
    print(f"  Normalized y_train - mean: {y_train_normalized.mean(axis=0)}, std: {y_train_normalized.std(axis=0)}")
else:
    y_train_normalized = y_train
    y_test_normalized = y_test
    y_train_mean = np.zeros(2)
    y_train_std = np.ones(2)
    print("\nℹ️  Output normalization: DISABLED")

# Use normalized data for training
X_train_final = X_train_normalized
X_test_final = X_test_normalized
y_train_final = y_train_normalized
y_test_final = y_test_normalized

print(f"\n📊 Final training data shapes:")
print(f"  X_train_final: {X_train_final.shape}")
print(f"  y_train_final: {y_train_final.shape}")

### Build Model

In [ ]:
# Build the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(input_dim,)))

# Add hidden layers
for i, hidden_size in enumerate(HIDDEN_LAYERS):
    if L2_REG > 0:
        model.add(tf.keras.layers.Dense(
            hidden_size, 
            activation=ACTIVATION,
            kernel_regularizer=tf.keras.regularizers.L2(L2_REG)
        ))
    else:
        model.add(tf.keras.layers.Dense(hidden_size, activation=ACTIVATION))
    
    if USE_BATCH_NORM:
        model.add(tf.keras.layers.BatchNormalization())
    
    if DROPOUT_RATE > 0:
        model.add(tf.keras.layers.Dropout(DROPOUT_RATE))

# Output layer (2 outputs: real and imaginary parts)
model.add(tf.keras.layers.Dense(2, activation=OUTPUT_ACTIVATION))

print("Model architecture:")
model.summary()

### Compile Model

In [ ]:
# Compile the model with AdamW optimizer
model.compile(
    optimizer=tf.keras.optimizers.AdamW(
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    ),
    loss=LOSS,
    metrics=['mae', 'mse']
)

print("✅ Model compiled")
print(f"  Optimizer: AdamW (lr={LEARNING_RATE}, weight_decay={WEIGHT_DECAY})")
print(f"  Loss: {LOSS}")
print(f"  Metrics: MAE, MSE")

### Setup Callbacks

In [ ]:
# Setup callbacks
callbacks = []

# Early stopping
if EARLY_STOPPING_PATIENCE > 0:
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    callbacks.append(early_stopping)
    print(f"Early stopping enabled (patience={EARLY_STOPPING_PATIENCE})")

# Learning rate reduction
if REDUCE_LR_PATIENCE > 0:
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=REDUCE_LR_FACTOR,
        patience=REDUCE_LR_PATIENCE,
        min_lr=1e-7,
        verbose=1
    )
    callbacks.append(reduce_lr)
    print(f"LR reduction enabled (patience={REDUCE_LR_PATIENCE}, factor={REDUCE_LR_FACTOR})")

print(f"\nTotal callbacks: {len(callbacks)}")

### Train Model

In [ ]:
# Train the model
print(f"\nTraining model on {len(X_train_final)} samples...\n")

history = model.fit(
    X_train_final,
    y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

## 9. Evaluate on Test Set

In [ ]:
# Evaluate on test set
print("Evaluating on test set...\n")

test_results = model.evaluate(X_test_final, y_test_final, verbose=0)

print(f"Test Results (on normalized data):")
print(f"  Loss ({LOSS}): {test_results[0]:.6f}")
print(f"  MAE: {test_results[1]:.6f}")
print(f"  MSE: {test_results[2]:.6f}")

# Make predictions on normalized inputs
y_pred_normalized = model.predict(X_test_final, verbose=0)

# Denormalize predictions and targets for metric calculation
if NORMALIZE_OUTPUTS:
    y_pred = y_pred_normalized * y_train_std + y_train_mean
    y_test_denorm = y_test  # Original unnormalized test data
    print("\n✅ Predictions denormalized for final metrics")
else:
    y_pred = y_pred_normalized
    y_test_denorm = y_test

# Calculate additional metrics on original scale
real_mae = np.mean(np.abs(y_test_denorm[:, 0] - y_pred[:, 0]))
imag_mae = np.mean(np.abs(y_test_denorm[:, 1] - y_pred[:, 1]))
real_mse = np.mean((y_test_denorm[:, 0] - y_pred[:, 0])**2)
imag_mse = np.mean((y_test_denorm[:, 1] - y_pred[:, 1])**2)

print(f"\nPer-component metrics (original scale):")
print(f"  Real part - MAE: {real_mae:.6f}, MSE: {real_mse:.6f}")
print(f"  Imag part - MAE: {imag_mae:.6f}, MSE: {imag_mse:.6f}")

# MAPE relative to mean of absolute values (since true mean is ~0)
real_mean_abs = np.mean(np.abs(y_test_denorm[:, 0]))
imag_mean_abs = np.mean(np.abs(y_test_denorm[:, 1]))

real_mape_rel = 100 * real_mae / real_mean_abs
imag_mape_rel = 100 * imag_mae / imag_mean_abs

print(f"\nMAPE relative to mean(|target|):")
print(f"  Real part - mean(|target|): {real_mean_abs:.6f}, MAPE: {real_mape_rel:.2f}%")
print(f"  Imag part - mean(|target|): {imag_mean_abs:.6f}, MAPE: {imag_mape_rel:.2f}%")

# Standard MAPE (only for non-zero targets)
real_nonzero_mask = np.abs(y_test_denorm[:, 0]) > 1e-10
imag_nonzero_mask = np.abs(y_test_denorm[:, 1]) > 1e-10

if np.sum(real_nonzero_mask) > 0:
    real_mape = 100 * np.mean(np.abs((y_test_denorm[real_nonzero_mask, 0] - y_pred[real_nonzero_mask, 0]) / y_test_denorm[real_nonzero_mask, 0]))
    print(f"\nStandard MAPE (element-wise, non-zero only):")
    print(f"  Real part - MAPE: {real_mape:.2f}% ({np.sum(real_nonzero_mask)}/{len(y_test)} non-zero)")

if np.sum(imag_nonzero_mask) > 0:
    imag_mape = 100 * np.mean(np.abs((y_test_denorm[imag_nonzero_mask, 1] - y_pred[imag_nonzero_mask, 1]) / y_test_denorm[imag_nonzero_mask, 1]))
    print(f"  Imag part - MAPE: {imag_mape:.2f}% ({np.sum(imag_nonzero_mask)}/{len(y_test)} non-zero)")

## 10. Visualize Results

### Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# MAE
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Training and Validation MAE', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

### Prediction vs Actual

In [ ]:
# Scatter plots: predicted vs actual (on original scale)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Calculate Pearson correlation coefficients
real_corr = np.corrcoef(y_test_denorm[:, 0], y_pred[:, 0])[0, 1]
imag_corr = np.corrcoef(y_test_denorm[:, 1], y_pred[:, 1])[0, 1]

# Real part
axes[0].scatter(y_test_denorm[:, 0], y_pred[:, 0], alpha=0.5, s=20)
min_real = min(y_test_denorm[:, 0].min(), y_pred[:, 0].min())
max_real = max(y_test_denorm[:, 0].max(), y_pred[:, 0].max())
axes[0].plot([min_real, max_real], [min_real, max_real], 'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual (Real Part)', fontsize=12)
axes[0].set_ylabel('Predicted (Real Part)', fontsize=12)
axes[0].set_title(f'Real Part: Predicted vs Actual\nMAE={real_mae:.6f}, PMCC={real_corr:.4f}', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')

# Imaginary part
axes[1].scatter(y_test_denorm[:, 1], y_pred[:, 1], alpha=0.5, s=20, color='orange')
min_imag = min(y_test_denorm[:, 1].min(), y_pred[:, 1].min())
max_imag = max(y_test_denorm[:, 1].max(), y_pred[:, 1].max())
axes[1].plot([min_imag, max_imag], [min_imag, max_imag], 'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual (Imag Part)', fontsize=12)
axes[1].set_ylabel('Predicted (Imag Part)', fontsize=12)
axes[1].set_title(f'Imaginary Part: Predicted vs Actual\nMAE={imag_mae:.6f}, PMCC={imag_corr:.4f}', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

### Error Distribution

In [ ]:
# Plot error distributions (on original scale)
real_errors = y_test_denorm[:, 0] - y_pred[:, 0]
imag_errors = y_test_denorm[:, 1] - y_pred[:, 1]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Real part errors
axes[0].hist(real_errors, bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(0, color='r', linestyle='--', linewidth=2, label='Zero Error')
axes[0].set_xlabel('Error (Actual - Predicted)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title(f'Real Part Error Distribution\nMean={np.mean(real_errors):.6f}, Std={np.std(real_errors):.6f}', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Imaginary part errors
axes[1].hist(imag_errors, bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1].axvline(0, color='r', linestyle='--', linewidth=2, label='Zero Error')
axes[1].set_xlabel('Error (Actual - Predicted)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title(f'Imaginary Part Error Distribution\nMean={np.mean(imag_errors):.6f}, Std={np.std(imag_errors):.6f}', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Summary

In [ ]:
print("="*70)
print("TRAINING SUMMARY")
print("="*70)

print(f"\nData:")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Input type: {INPUT_TYPE}")
print(f"  Input dimension: {input_dim}")
print(f"  Output dimension: 2 (real + imag)")

print(f"\nModel:")
print(f"  Architecture: {input_dim} -> {' -> '.join(map(str, HIDDEN_LAYERS))} -> 2")
print(f"  Total parameters: {model.count_params():,}")
print(f"  Activation: {ACTIVATION}")
print(f"  Learning rate: {LEARNING_RATE}")

print(f"\nTraining:")
print(f"  Epochs run: {len(history.history['loss'])}")
print(f"  Final training loss: {history.history['loss'][-1]:.6f}")
print(f"  Final validation loss: {history.history['val_loss'][-1]:.6f}")

print(f"\nTest Performance:")
print(f"  Overall MAE: {test_results[1]:.6f}")
print(f"  Real part MAE: {real_mae:.6f}")
print(f"  Imag part MAE: {imag_mae:.6f}")

print("\n" + "="*70)